# Μέρος Β1.iii: Ταξινόμηση Νομικών Εγγράφων με XGBoost και Word2Vec

Αυτό το notebook αφορά την υλοποίηση και αξιολόγηση μοντέλων XGBoost για την ταξινόμηση ελληνικών νομικών εγγράφων, χρησιμοποιώντας πυκνές αναπαραστάσεις κειμένου (dense embeddings) που προέρχονται από το Word2Vec. Η εργασία αυτή αποτελεί μέρος του ερωτήματος Β1.iii της εξαμηνιαίας εργασίας.

**Επεξήγηση Κελιού:**
Το παραπάνω κελί Markdown εισάγει τον σκοπό του notebook: την εφαρμογή XGBoost με Word2Vec embeddings για την ταξινόμηση νομικών κειμένων, σύμφωνα με τις απαιτήσεις του Β1.iii.

## 1. Σύντομη Επισκόπηση: XGBoost με Word2Vec

Το XGBoost (Extreme Gradient Boosting) είναι ένας ισχυρός και ευρέως χρησιμοποιούμενος αλγόριθμος μηχανικής μάθησης που βασίζεται στην τεχνική gradient boosting. Είναι γνωστός για την υψηλή του απόδοση και την ταχύτητά του.

Σε αυτό το πλαίσιο, συνδυάζουμε το XGBoost με Word2Vec embeddings:
1.  **Word2Vec Embeddings:** Τα νομικά κείμενα μετατρέπονται πρώτα σε πυκνά διανύσματα (document embeddings) χρησιμοποιώντας ένα προ-εκπαιδευμένο ή εκπαιδευμένο επί τόπου μοντέλο Word2Vec. Συνήθως, αυτό γίνεται παίρνοντας τον μέσο όρο των word embeddings των λέξεων κάθε εγγράφου.
2.  **XGBoost Classifier:** Αυτά τα document embeddings χρησιμοποιούνται στη συνέχεια ως χαρακτηριστικά εισόδου για την εκπαίδευση ενός ταξινομητή XGBoost.

Ο συνδυασμός αυτός αξιοποιεί την ικανότητα του Word2Vec να συλλαμβάνει σημασιολογικές πληροφορίες και την ισχύ του XGBoost στην ταξινόμηση.

**Επεξήγηση Κελιού:**
Αυτό το κελί Markdown παρέχει μια πολύ σύντομη επισκόπηση του XGBoost και του πώς συνδυάζεται με το Word2Vec για την ταξινόμηση κειμένου.

## 2. Υλοποίηση και Πειράματα

**Επεξήγηση Κελιού:**
Αυτό το κελί Markdown σηματοδοτεί την έναρξη της ενότητας υλοποίησης και πειραμάτων.

In [1]:
import sys
import os
import numpy as np
import xgboost as xgb # Βιβλιοθήκη XGBoost

# Προσθήκη του γονικού καταλόγου στο path για την εισαγωγή του utils.py
current_notebook_dir = os.getcwd()
parent_dir = os.path.dirname(current_notebook_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

try:
    from utils import (
        load_and_preprocess_data,
        run_experiment, # Αυτή η συνάρτηση χειρίζεται και το Word2Vec feature generation
        script_execution_timer
    )
    print("Successfully imported from utils.py")
except ImportError as e:
    print(f"Error importing from utils.py: {e}")
    print(f"Please ensure utils.py is in the correct path: {parent_dir} or that the notebook's parent directory is correctly identified.")

# Η import της Word2Vec γίνεται μέσα στο utils.py όταν χρειάζεται

c:\Users\USER\Desktop\PROJECTS\ML_GreekLegalDocs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Successfully imported from utils.py


**Επεξήγηση Κελιού:**
Το παραπάνω κελί κώδικα εισάγει τις απαραίτητες βιβλιοθήκες Python (sys, os, numpy, xgboost) και ρυθμίζει το `sys.path` για την εισαγωγή των συναρτήσεων από το `utils.py`.

### 2.1. Ρυθμίσεις Πειράματος

Παρακαλώ επιλέξτε τις παραμέτρους για το πείραμα.

**Επεξήγηση Κελιού:**
Αυτό το κελί Markdown εισάγει την ενότητα όπου ο χρήστης μπορεί να διαμορφώσει τις παραμέτρους του πειράματος.

In [2]:
# --- Configuration ---
# Επιλέξτε την ετικέτα-στόχο: "volume", "chapter", ή "subject"
DATASET_CONFIG = "volume" 

# Ποσοστό του dataset που θα χρησιμοποιηθεί
SUBSET_PERCENTAGE = 0.1 # Για λόγους ταχύτητας. Αλλάξτε σε 1.0 για πλήρη δεδομένα.

# Αριθμός K-folds για Cross-Validation (1 για απλό train/test split)
N_SPLITS_CV = 1 
if DATASET_CONFIG == "volume":
    print("INFO: For 'volume' target, typically 5-fold CV was used in the report. Adjust N_SPLITS_CV if needed.")
    # N_SPLITS_CV = 5 # Uncomment to match report for 'volume'

RANDOM_STATE = 42
SINGLE_SPLIT_TEST_SIZE = 0.2

SAVE_TRAINED_FEATURE_MODELS = True
LOAD_TRAINED_FEATURE_MODELS_IF_EXIST = True

# Word2Vec parameters (όπως στην αναφορά)
WORD2VEC_VECTOR_SIZE = 100
WORD2VEC_WINDOW = 5
WORD2VEC_MIN_COUNT = 2
WORD2VEC_WORKERS = 4 
WORD2VEC_SG = 0      # CBOW
WORD2VEC_EPOCHS = 10

# XGBoost parameters (όπως στην αναφορά)
XGB_N_ESTIMATORS = 100
XGB_LEARNING_RATE = 0.1
XGB_MAX_DEPTH = 3
XGB_EVAL_METRIC = 'mlogloss' # mlogloss για multi-class classification
# --- End Configuration ---

INFO: For 'volume' target, typically 5-fold CV was used in the report. Adjust N_SPLITS_CV if needed.


**Επεξήγηση Κελιού:**
Αυτό το κελί κώδικα περιέχει τις παραμέτρους διαμόρφωσης για το πείραμα XGBoost με Word2Vec, συμπεριλαμβανομένων των ρυθμίσεων για το dataset, το Word2Vec και το XGBoost.

In [3]:
@script_execution_timer
def run_xgboost_w2v_experiment(dataset_config, subset_percentage, n_splits_cv, random_state, 
                               single_split_test_size, save_feature_models, load_feature_models,
                               w2v_vector_size, w2v_window, w2v_min_count, w2v_workers, w2v_sg, w2v_epochs,
                               xgb_n_estimators, xgb_learning_rate, xgb_max_depth, xgb_eval_metric):
    model_name_script = "XGBoost"
    feature_method_name = "Word2Vec"
    
    base_run_id = f"{model_name_script}_{feature_method_name}_{dataset_config}"
    
    print(f"Starting {model_name_script} with {feature_method_name} for '{dataset_config}' config...")
    
    feature_config = {
        'method': 'word2vec',
        'params': {
            'vector_size': w2v_vector_size,
            'window': w2v_window,
            'min_count': w2v_min_count,
            'workers': w2v_workers,
            'sg': w2v_sg,
            'epochs': w2v_epochs,
            'seed': random_state
        }
    }
    
    model_class = xgb.XGBClassifier
    model_init_params = {
        'n_estimators': xgb_n_estimators,
        'learning_rate': xgb_learning_rate,
        'max_depth': xgb_max_depth,
        'eval_metric': xgb_eval_metric,
        'random_state': random_state,
        # 'use_label_encoder': False # For newer XGBoost versions, this is often handled automatically or not needed if labels are 0 to N-1
    }

    script_dir = os.getcwd()
    experiment_output_base_dir = os.path.join(script_dir, "outputs", base_run_id)
    reports_output_dir = os.path.join(experiment_output_base_dir, "reports")
    feature_models_output_dir = os.path.join(experiment_output_base_dir, "feature_models")
    
    os.makedirs(reports_output_dir, exist_ok=True)
    os.makedirs(feature_models_output_dir, exist_ok=True)
    print(f"INFO: Experiment outputs will be saved in '{experiment_output_base_dir}'")

    if 0 < subset_percentage < 1.0:
        print(f"INFO: Using a {subset_percentage*100:.0f}% subset of the data.")
    else:
        print("INFO: Using all available data (after filtering).")
    
    if n_splits_cv == 1:
        print(f"INFO: Performing a single train/test split (test_size={single_split_test_size}).")
    elif n_splits_cv > 1:
        print(f"INFO: Performing {n_splits_cv}-Fold Cross-Validation.")
    else:
        print("ERROR: N_SPLITS_CV must be >= 1.")
        return

    texts_proc, labels_proc, unique_labels_proc, unique_labels_proc_str = load_and_preprocess_data(
        dataset_config, subset_percentage, random_state
    )

    if texts_proc is None or len(texts_proc) == 0:
        print("Failed to load or preprocess data. Exiting.")
        return

    success = run_experiment(
        texts_to_process=texts_proc,
        labels_to_process=labels_proc,
        unique_labels_to_process=unique_labels_proc,
        unique_labels_to_process_str=unique_labels_proc_str,
        n_splits=n_splits_cv,
        feature_config=feature_config,
        model_class=model_class,
        model_init_params=model_init_params,
        base_run_identifier=base_run_id, 
        reports_output_dir=reports_output_dir,
        feature_models_output_dir=feature_models_output_dir,
        random_state=random_state,
        save_trained_features=save_feature_models,
        load_trained_features_if_exist=load_feature_models,
        single_split_test_size=single_split_test_size
    )

    if success:
        print(f"\nExperiment '{base_run_id}' completed successfully.")
    else:
        print(f"\nExperiment '{base_run_id}' encountered errors.")

    print(f"\n{model_name_script} with {feature_method_name} script finished for {dataset_config} dataset.")

**Επεξήγηση Κελιού:**
Αυτό το κελί κώδικα ορίζει τη συνάρτηση `run_xgboost_w2v_experiment`. Αυτή η συνάρτηση ενσωματώνει τη λογική για την εκτέλεση ενός πειράματος XGBoost με Word2Vec, καλώντας τις γενικές συναρτήσεις από το `utils.py` για τη διαχείριση δεδομένων, χαρακτηριστικών και εκτέλεσης του μοντέλου.

### 2.2. Εκτέλεση Πειράματος

Το παρακάτω κελί θα εκτελέσει το πείραμα.

**Επεξήγηση Κελιού:**
Αυτό το κελί Markdown εισάγει το κελί κώδικα που θα εκτελέσει το πείραμα.

In [4]:
run_xgboost_w2v_experiment(
    dataset_config=DATASET_CONFIG,
    subset_percentage=SUBSET_PERCENTAGE,
    n_splits_cv=N_SPLITS_CV,
    random_state=RANDOM_STATE,
    single_split_test_size=SINGLE_SPLIT_TEST_SIZE,
    save_feature_models=SAVE_TRAINED_FEATURE_MODELS,
    load_feature_models=LOAD_TRAINED_FEATURE_MODELS_IF_EXIST,
    w2v_vector_size=WORD2VEC_VECTOR_SIZE,
    w2v_window=WORD2VEC_WINDOW,
    w2v_min_count=WORD2VEC_MIN_COUNT,
    w2v_workers=WORD2VEC_WORKERS,
    w2v_sg=WORD2VEC_SG,
    w2v_epochs=WORD2VEC_EPOCHS,
    xgb_n_estimators=XGB_N_ESTIMATORS,
    xgb_learning_rate=XGB_LEARNING_RATE,
    xgb_max_depth=XGB_MAX_DEPTH,
    xgb_eval_metric=XGB_EVAL_METRIC
)


--- Script 'run_xgboost_w2v_experiment' started at: 2025-05-24 20:30:39 ---
Starting XGBoost with Word2Vec for 'volume' config...
INFO: Experiment outputs will be saved in 'c:\Users\USER\Desktop\PROJECTS\ML_GreekLegalDocs\merosB1\outputs\XGBoost_Word2Vec_volume'
INFO: Using a 10% subset of the data.
INFO: Performing a single train/test split (test_size=0.2).
Loading dataset 'AI-team-UoA/greek_legal_code' with 'volume' configuration...
Dataset loaded in 6.66 seconds.
Extracting text and labels...
Full dataset extracted in 0.52s. Samples: 28536
Original unique labels in full dataset (volume config): 47
Filtering out classes with fewer than 2 samples...
No classes needed filtering based on min_samples_per_class.
Dataset size after filtering for min_samples_per_class: 28536 samples.
Unique labels after initial filtering: 47

Selecting a 10% subset from the filtered data...
Subset size: 2853
Unique labels in data finally selected for processing: 47

--- Performing single train/test split (

**Επεξήγηση Κελιού:**
Αυτό το κελί κώδικα εκτελεί το πείραμα καλώντας τη συνάρτηση `run_xgboost_w2v_experiment` με τις παραμέτρους που ορίστηκαν προηγουμένως.

## 3. Αποτελέσματα και Σχολιασμός

Τα αναλυτικά αποτελέσματα (classification reports) αποθηκεύονται στον κατάλογο `outputs/[MODEL_FEATURE_CONFIG]/reports/`.

Για παράδειγμα:
`merosB1/iii/outputs/XGBoost_Word2Vec_subject/reports/`

Τα αποτελέσματα στην επιστημονική αναφορά (`B1.md`) για XGBoost+W2V προέρχονται από εκτελέσεις αυτού του κώδικα.

**Επεξήγηση Κελιού:**
Αυτό το τελικό κελί Markdown παρέχει πληροφορίες για το πού αποθηκεύονται τα αποτελέσματα του πειράματος.